In [1]:
import numpy as np
import os
import pandas as pd
import SimpleITK as sitk
main_dir="/root/data/post_norm_min_max_bias_on"
train_folds_path="/workspaces/prostate_opi_year_3/data/AI4AR_uczacy_foldy.csv"
test_ids_path="/workspaces/prostate_opi_year_3/data/AI4ARtestowy.csv"

folds_df=pd.read_csv(train_folds_path)
test_ids_df=pd.read_csv(test_ids_path,header=None)

test_ids = test_ids_df.iloc[:, 0].to_numpy()

folds_df.columns

Index(['Patient ID', 'fold'], dtype='object')

In [2]:
example_path="/root/data/post_norm_min_max_bias_on/005/5_empty.nii.gz"
image=sitk.ReadImage(example_path)
sitk.GetArrayFromImage(image)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [3]:
import os
import pandas as pd
import yaml
import json
from monai.apps.auto3dseg.auto_runner import AutoRunner

main_dir = "/root/data/post_norm_min_max_bias_on"
train_folds_path = "/workspaces/prostate_opi_year_3/data/AI4AR_uczacy_foldy.csv"
test_ids_path = "/workspaces/prostate_opi_year_3/data/AI4ARtestowy.csv"

# Load the data
folds_df = pd.read_csv(train_folds_path)
test_ids_df = pd.read_csv(test_ids_path, header=None)
test_ids = test_ids_df.iloc[:, 0].to_numpy()

# def create_empty_label_if_not_exists(image_path, label_path):
#     # Check if the label file already exists
#     if not os.path.exists(label_path):
#         # Read the image from image_path
#         image = sitk.ReadImage(image_path)
        
#         # Get the size and spacing of the original image
#         size = image.GetSize()
#         spacing = image.GetSpacing()
#         origin = image.GetOrigin()
#         direction = image.GetDirection()
        
#         # Create a new image with the same size, spacing, origin, and direction
#         # but with all pixel values set to zero and of type uint16
#         empty_label = sitk.Image(size, sitk.sitkUInt16)
#         empty_label.SetSpacing(spacing)
#         empty_label.SetOrigin(origin)
#         empty_label.SetDirection(direction)
        
#         # Save the new image to label_path
#         sitk.WriteImage(empty_label, label_path)
#         print(f"Created empty label image at {label_path}")
#     else:
#         # print(f"Label image already exists at {label_path}")
#         pass

# Prepare the data dictionary
data_dict = {
    "training": [],
    "testing": [],
    "modality": "MRI"
}
def modify_string(string):
    if len(string) >=3:
        return string
    elif len(string) == 2:
        return "0" + string
    else:
        return "00" + string

# Populate the training data
for _, row in folds_df.iterrows():
    patient_id = row['Patient ID']
    fold = row['fold']
    ppath_folder=modify_string(str(patient_id))
    image_path = os.path.join(main_dir, ppath_folder, f"{str(patient_id)}_adc.nii.gz")
    label_path = os.path.join(main_dir,ppath_folder, f"{str(patient_id)}_lesion_union_adc.nii.gz")
    if not os.path.exists(label_path):
        label_path = os.path.join(main_dir,ppath_folder, f"{str(patient_id)}_empty.nii.gz")
    data_dict["training"].append({
        "fold": int(fold)-1,
        "image": image_path,
        "label": label_path
    })

# Populate the testing data
for test_id in test_ids:
    image_path = os.path.join(main_dir, str(test_id), "str(test_id)_adc.nii.gz")
    data_dict["testing"].append({
        "image": image_path
    })

# Save the data dictionary to a YAML file
datalist_path = "/workspaces/prostate_opi_year_3/data/input.json"
with open(datalist_path, 'w') as yaml_file:
    json.dump(data_dict, yaml_file)


# Define the work directory
work_dir = "/workspaces/prostate_opi_year_3/data/work_dir"
os.makedirs(work_dir, exist_ok=True)

runner = AutoRunner(
    work_dir=work_dir,
    input={
        "modality": "MRI",
        "datalist": datalist_path,
        "dataroot": work_dir,
    },
)
# Prepare the configuration dictionary
# config_dict = {
#     "datalist": input_yaml_path,
#     "dataroot": main_dir,
#     "modality": "MRI"
# }

# # Save the configuration dictionary to a YAML file
# config_yaml_path = os.path.join(work_dir, "config.json")
# with open(config_yaml_path, 'w') as yaml_file:
#     json.dump(config_dict, yaml_file)

# print(f"Config YAML file saved to {config_yaml_path}")

# Initialize and run the AutoRunner
# runner = AutoRunner(work_dir=work_dir, input=config_yaml_path)
runner.run()


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-08-21 11:50:25,476 - INFO - AutoRunner using work directory /workspaces/prostate_opi_year_3/data/work_dir
2024-08-21 11:50:25,497 - INFO - Datalist was copied to work_dir: /workspaces/prostate_opi_year_3/data/work_dir/input.json
2024-08-21 11:50:25,499 - INFO - Found num_fold 5 based on the input datalist /workspaces/prostate_opi_year_3/data/work_dir/input.json.
2024-08-21 11:50:25,499 - INFO - Setting num_fold 5 based on the input datalist /workspaces/prostate_opi_year_3/data/work_dir/input.json.
2024-08-21 11:50:25,501 - INFO - Using user defined command running prefix , will override other settings
2024-08-21 11:50:25,502 - INFO - Skipping data analysis...
2024-08-21 11:50:25,502 - INFO - Skipping algorithm generation...
2024-08-21 11:50:25,503 - INFO - Skipping algorithm training...
2024-08-21 11:50:25,503 - INFO - Ensembling using 2 GPU!
2024-08-21 11:50:25,504 - INFO - ['torchrun', '--nnodes', '1', '--nproc_per_node', '2', '-m', 'monai.apps.auto3dseg', 'EnsembleRunner', 'ens

KeyboardInterrupt: 

@workspace You are Python expert. Prepare necessary files and code to execute "auto3dseg" on supplied dataset. 
Dataset is located at """main_dir""" inside it each folder name is a "Patient Id" . Inside each patient folder label is called "1_lesion_union_adc.nii.gz" and image is called "1_adc.nii.gz" . Test ids are present in array "test_ids". Patient id's are in dataframe """test_ids_df""" which has 2 columns """['Patient ID', 'fold']""" where fold is number from 1 to 5 and indicate the fold on which giben Patient ID should be in validation group. 

Do this by correcting the code """
import os
import pandas as pd
import yaml
from monai.apps.auto3dseg.auto_runner import AutoRunner

main_dir = "/root/data/post_norm_min_max_bias_on"
train_folds_path = "/workspaces/prostate_opi_year_3/data/AI4AR_uczacy_foldy.csv"
test_ids_path = "/workspaces/prostate_opi_year_3/data/AI4ARtestowy.csv"

# Load the data
folds_df = pd.read_csv(train_folds_path)
test_ids_df = pd.read_csv(test_ids_path, header=None)
test_ids = test_ids_df.iloc[:, 0].to_numpy()

# Prepare the data dictionary
data_dict = {
    "training": [],
    "testing": [],
    "modality": "MRI"
}

# Populate the training data
for _, row in folds_df.iterrows():
    patient_id = row['Patient ID']
    fold = row['fold']
    image_path = os.path.join(main_dir, str(patient_id), "1_adc.nii.gz")
    label_path = os.path.join(main_dir, str(patient_id), "1_lesion_union_adc.nii.gz")
    data_dict["training"].append({
        "fold": int(fold)-1,
        "image": image_path,
        "label": label_path
    })

# Populate the testing data
for test_id in test_ids:
    image_path = os.path.join(main_dir, str(test_id), "1_adc.nii.gz")
    data_dict["testing"].append({
        "image": image_path
    })

# Save the data dictionary to a YAML file
input_yaml_path = "/workspaces/prostate_opi_year_3/data/input.yaml"
with open(input_yaml_path, 'w') as yaml_file:
    yaml.dump(data_dict, yaml_file)

print(f"Input YAML file saved to {input_yaml_path}")

# Define the work directory
work_dir = "/workspaces/prostate_opi_year_3/data/work_dir"
os.makedirs(work_dir, exist_ok=True)

# Prepare the configuration dictionary
config_dict = {
    "datalist": input_yaml_path,
    "dataroot": main_dir,
    "modality": "MRI"
}

# Save the configuration dictionary to a YAML file
config_yaml_path = os.path.join(work_dir, "config.yaml")
with open(config_yaml_path, 'w') as yaml_file:
    yaml.dump(config_dict, yaml_file)

print(f"Config YAML file saved to {config_yaml_path}")

# Initialize and run the AutoRunner
runner = AutoRunner(work_dir=work_dir, input=config_yaml_path)
runner.run()
"""

Which currently give error """KeyError: 'training'""" Modality is MRI

Now take a deep breath reread your answer and make sure that code is correct and that it will work as intended. 